## Browser Side

In [ ]:
cat << EOF > ~/digitypography/themes/freemind/source/js/twitter.js
var twitter = (function() {
  function escapeHtml(str) {
    return \$('<div/>').text(str).html();
  }
  function render(target, tweets) {
    var i = 0, fragment = '', t = \$(target)[0];

    for(i = 0; i < tweets.length; ++i) {
      fragment += '<li><i class="fa fa-star"></i><a href="https://twitter.com/charisbit/status/' + tweets[i].id + '">'
          + tweets[i].created_at.replace('T', '　').replace('.000Z', '　UTC') + '</a><p>' + escapeHtml(tweets[i].text||'') + '</p></li>';
    }
    t.innerHTML = fragment;
  }
  return {
    showTweets: function(options) {
      options.blacklist = options.blacklist.split(',');
      \$.ajax({
          url: 'https://jhub.name/typography/?tweet.fields=created_at&max_results=' + options.count
        , dataType: 'json'
        , error: function (err) { \$(options.target + ' li.loading').addClass('error').text('Error loading feed'); }
        , success: function (data) {
          if (!data ) { return; }
          var tweets = [];
          for (var i = 0; i < data.data.length; ++i) {
            if (options.blacklist instanceof Array && options.blacklist.indexOf(data.data[i].id) !== -1) { continue; }
            tweets.push(data.data[i]);
          }
          render(options.target, tweets);
        }
      });
    }
  };
})();
EOF

## Server Side

* https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference

In [ ]:
curl -H "Authorization: Bearer ${BEARER_TOKEN}" 'https://api.twitter.com/2/users/by/username/charisbit?user.fields=created_at'

{"data":{"name":"Charis","username":"charisbit","created_at":"2022-02-28T13:05:23.000Z","id":"1498283136295211010"}}

* https://developer.twitter.com/en/docs/twitter-api/tweets/timelines/api-reference/get-users-id-tweets

In [ ]:
curl -H "Authorization: Bearer ${BEARER_TOKEN}" 'https://api.twitter.com/2/users/1498283136295211010/tweets?tweet.fields=created_at&max_results=10'

{"meta":{"result_count":0}}

* My solution using [Traefik](https://doc.traefik.io/traefik/v1.7/basics/#custom-headers) to proxy Tweets API

In [ ]:
cat << EOF | sudo tee /opt/tljh/state/dynamic/typography.toml > /dev/null

[frontends.typography]
backend = "typography"

[frontends.typography.routes.r1]
rule = "PathPrefix:/typography/"

[frontends.typography.routes.r2]
rule = "ReplacePath:/2/users/1498283136295211010/tweets"

[frontends.typography.headers.customrequestheaders]
Authorization = "Bearer ${BEARER_TOKEN}"

[frontends.typography.headers.customresponseheaders]
Access-Control-Allow-Origin = "*"

[backends.typography.servers.s1]
url = "https://api.twitter.com"
weight = 1

EOF

In [ ]:
sudo systemctl restart traefik

In [ ]:
curl 'https://jhub.name/typography/?tweet.fields=created_at&max_results=10'

{"data":[{"created_at":"2022-02-28T14:15:00.000Z","id":"1498300708893835264","text":"$GoodBit is designed to measure the meaningfulness for bits. The average $GoodBit of each bit of contents of NFTs of corresponding Copy and Quote License Tokens is stabilized at 1 $GoodBit, so that 1 $GoodBit can be used to pay for copying and quoting 1 bit on average."}],"meta":{"oldest_id":"1498300708893835264","newest_id":"1498300708893835264","result_count":1}}